In [ ]:
# import
import pandas as pd

In [ ]:
#load csv and convert few datetime objects
fact =pd.read_csv("olist_fact_sales.csv",parse_dates=['order_purchase_timestamp',
                                                                              'order_approved_at',
                                                                              'order_delivered_carrier_date',
                                                                              'order_delivered_customer_date',
                                                                              'order_estimated_delivery_date'],encoding='utf-8')

In [ ]:
#drop unwanted column
fact.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
# reference date
ref_date = fact['order_purchase_timestamp'].max() + pd.Timedelta(days=1)

In [ ]:
# RFM analysis
rfm=(fact.groupby('customer_id').agg(Recency=('order_purchase_timestamp',lambda x: (ref_date - x.max()).days),
                               Frequency =('order_id','nunique'),
                               Monetary=('total_payment_value', 'sum')).reset_index())

In [ ]:
#sanity check
rfm.describe()

In [ ]:
# customer segments
rfm['R_score'] =pd.qcut(x=rfm['Recency'],q=5,labels=range(5,0,-1))
rfm['M_score']=pd.qcut(x=rfm['Monetary'],q=5,labels=range(1,6,1))
rfm['F_score'] = 1

def rm_segment(row):
    if row['R_score'] >= 4 and row['M_score'] >= 4:
        return 'Champions'
    elif row['R_score'] >= 4 and row['M_score'] <= 3:
        return 'New Customers'
    elif row['R_score'] <= 2 and row['M_score'] >= 4:
        return 'At Risk High Value'
    elif row['R_score'] <= 2 and row['M_score'] <= 2:
        return 'Low Value'
    else:
        return 'Regular'

rfm['customer_segment'] = rfm.apply(rm_segment,axis=1)

In [ ]:
# sanity check
rfm['customer_segment'].value_counts()

In [ ]:
# sanity check 
rfm.groupby('customer_segment')[['Recency','Monetary']].mean()

In [ ]:
# export  file

rfm.to_csv("olist_RFM_analysis.csv")
